In [1]:
import pandas as pd
import sqlalchemy
import sqlite3

#### 1. List all the directors who directed a 'Comedy' movie in a leap year. (You need to check that the genre is 'Comedy’ and year is a leap year) Your query should return director name, the movie name, and the year.

In [95]:
d1 = pd.read_sql_query('''
SELECT Person.Name, title, year
FROM
(SELECT * 
FROM Movie
WHERE (year % 4 = 0) and (year % 100 != 0) or (year % 400 = 0)) 
JOIN M_Genre USING (MID)
JOIN Genre USING (GID)
JOIN M_Director USING (MID)
JOIN Person USING (PID)
WHERE Genre.Name LIKE '%Comedy%' 

''', con)
print(d1.sample(n=5))

                 Name              title  year
113    Sanjeev Sharma      Saat Uchakkey  2016
389  Ganapathy Bharat            Hari Om  2004
153      Rohit Shetty    Golmaal Returns  2008
359              Brij   Bombay 405 Miles  1980
370      Govind Menon  Kis Kis Ki Kismat  2004


#### 2. List the names of all the actors who played in the movie 'Anand' (1971)

In [12]:
con = sqlite3.connect('Db-IMDB.db')
d2 = pd.read_sql_query('''

SELECT Person.Name FROM
(SELECT *
FROM M_Cast
WHERE M_Cast.MID =
(SELECT MID
FROM Movie
WHERE title LIKE 'Anand'))
JOIN Person
USING (PID)


''', con)
print(d2)

                 Name
0       Rajesh Khanna
1    Amitabh Bachchan
2       Sumita Sanyal
3          Ramesh Deo
4           Seema Deo
5      Asit Kumar Sen
6          Dev Kishan
7        Atam Prakash
8       Lalita Kumari
9              Savita
10     Brahm Bhardwaj
11       Gurnam Singh
12       Lalita Pawar
13        Durga Khote
14         Dara Singh
15      Johnny Walker
16          Moolchand


#### 3. List all the actors who acted in a film before 1970 and in a film after 1990. (That is: < 1970 and > 1990.)

In [ ]:
#Removing spaces from PID column
con = sqlite3.connect('Db-IMDB.db', timeout=10)
sql = '''UPDATE M_Cast SET PID=Trim(PID)''' 
cur = con.cursor() 
cur.execute(sql) 
con.commit()

In [13]:
con = sqlite3.connect('Db-IMDB.db', timeout=10)
d3 = pd.read_sql_query('''
SELECT Person.Name FROM
(SELECT * FROM
(SELECT *
FROM M_Cast
JOIN
(SELECT * 
FROM Movie
WHERE year < 1970)
USING(MID))
JOIN
(SELECT *
FROM M_Cast
JOIN
(SELECT * 
FROM Movie
WHERE year > 1990)
USING(MID)) 
USING (PID))
JOIN Person USING (PID)
''', con)
print(d3.sample(n=5))

                   Name
8267          Ram Mohan
4979   Amitabh Bachchan
6928        Sanjay Dutt
5074        Dina Pathak
1296       Rishi Kapoor


#### 4. List all directors who directed 10 movies or more, in descending order of the number of movies they directed. Return the directors' names and the number of movies each of them directed.

In [61]:
con = sqlite3.connect('Db-IMDB.db', timeout=10)
sql = '''UPDATE Person SET Name=Trim(Name)''' 
cur = con.cursor() 
cur.execute(sql) 
con.commit()

In [62]:
con = sqlite3.connect('Db-IMDB.db')
d4 = pd.read_sql_query('''
SELECT DISTINCT Name, cnt FROM
(SELECT * FROM
(SELECT * 
FROM
(SELECT DISTINCT PID, count(PID) AS cnt 
FROM M_Director 
GROUP by PID))
WHERE cnt>10)
JOIN Person
USING (PID) 
ORDER BY cnt DESC
''', con)
print(d4.head(6))

                   Name  cnt
0          David Dhawan   39
1          Mahesh Bhatt   35
2       Ram Gopal Varma   30
3          Priyadarshan   30
4          Vikram Bhatt   29
5  Hrishikesh Mukherjee   27


#### 5.a. For each year, count the number of movies in that year that had only female actors.

In [60]:
con = sqlite3.connect('Db-IMDB.db')
d5 = pd.read_sql_query('''
SELECT year, COUNT(year) FROM Movie WHERE MID NOT IN (SELECT MID FROM M_Cast JOIN Person USING (PID) WHERE Gender = 'Male') 
GROUP BY year
''', con)
print(d5.head())
print(d5.shape)

   year  COUNT(year)
0  1939            1
1  1999            1
2  2000            1
3  2009            1
4  2012            1
5  2018            2
(6, 2)


#### 5.b. Now include a small change: report for each year the percentage of movies in that year with only female actors, and the total number of movies made that year. For example, one answer will be: 1990 31.81 13522 meaning that in 1990 there were 13,522 movies, and 31.81% had only female actors. You do not need to round your answer.

In [74]:
con = sqlite3.connect('Db-IMDB.db')
d5 = pd.read_sql_query('''
SELECT year, cnt_females, cnt_total, ((cnt_females*100.00)/cnt_total) perc_females FROM
(SELECT year, COUNT(year) AS cnt_females FROM Movie WHERE MID NOT IN (SELECT MID FROM M_Cast JOIN Person USING (PID) WHERE Gender = 'Male') 
GROUP BY year)
JOIN
(SELECT year, COUNT(year) AS cnt_total FROM Movie 
GROUP BY year)
USING (year)
''', con)
print(d5)

   year  cnt_females  cnt_total  perc_females
0  1939            1          2     50.000000
1  1999            1         66      1.515152
2  2000            1         64      1.562500
3  2009            1        110      0.909091
4  2012            1        111      0.900901
5  2018            2        104      1.923077


#### Find the film(s) with the largest cast. Return the movie title and the size of the cast. By "cast size" we mean the number of distinct actors that played in that movie: if an actor played multiple roles, or if it simply occurs multiple times in casts, we still count her/him only once.

In [82]:
con = sqlite3.connect('Db-IMDB.db')
d6 = pd.read_sql_query('''
SELECT MID, COUNT(MID) AS cast_cnt
FROM M_Cast
GROUP BY MID
ORDER BY cast_cnt DESC LIMIT 5
''', con)
print(d6)

         MID  cast_cnt
0  tt5164214       238
1  tt0451631       233
2  tt6173990       215
3  tt1188996       213
4  tt3498820       191


#### A decade is a sequence of 10 consecutive years. For example, say in your database you have movie information starting from 1965. Then the first decade is 1965, 1966, ..., 1974; the second one is 1967, 1968, ..., 1976 and so on. Find the decade D with the largest number of films and the total number of films in D. 

In [4]:
con = sqlite3.connect('Db-IMDB.db')
d7 = pd.read_sql_query('''
SELECT y.year AS decade_start, y.year + 9 AS decade_end,
COUNT(*) AS num_movies
FROM
(SELECT DISTINCT year FROM Movie) AS y JOIN
Movie AS m
ON
m.year >= y.year and m.year < y.year + 10
GROUP BY y.year
ORDER BY COUNT(*) DESC
''', con)
print(d7.head())
print(d7.tail())

  decade_start  decade_end  num_movies
0         2008        2017        1205
1         2009        2018        1202
2         2007        2016        1188
3         2005        2014        1170
4         2006        2015        1160
   decade_start  decade_end  num_movies
73         1943        1952          25
74         1941        1950          14
75         1939        1948          11
76         1936        1945           7
77         1931        1940           6


#### Find the actors that were never unemployed for more than 3 years at a stretch. (Assume that the actors remain unemployed between two consecutive movies).

In [12]:
con = sqlite3.connect('Db-IMDB.db')
d7 = pd.read_sql_query('''
select m.*, (m.next_year - m.year) as gap
from
(
select pid, name, title, year,
LEAD(year, 1, 0) OVER (PARTITION BY name ORDER BY year ASC) AS next_year
from 
(
SELECT pid, name, title, year
FROM
(SELECT Name, PID, MID, title, year
FROM
Movie JOIN
(SELECT * FROM 
Person JOIN M_Cast
USING(PID))
USING(MID))
)
)m
where m.next_year > 0 and (m.next_year - m.year) < 3
''', con)
print(d7.head(5))

         pid             name            title  year next_year  gap
0  nm1869655  A. Abdul Hameed       Prem Nagar  1974      1975    1
1  nm0359845      A.K. Hangal     Teesri Kasam  1966      1967    1
2  nm0359845      A.K. Hangal          Shagird  1967      1969    2
3  nm0359845      A.K. Hangal  Saat Hindustani  1969      1971    2
4  nm0359845      A.K. Hangal            Guddi  1971      1971    0


#### Find all the actors that made more movies with Yash Chopra than any other director.

In [2]:
con = sqlite3.connect('Db-IMDB.db')
d7 = pd.read_sql_query('''
WITH
YASH_CHOPRAS_PID AS
(
SELECT TRIM(P.PID) AS PID
FROM Person P
WHERE Trim(P.Name) = 'Yash Chopra'
),
NUM_OF_MOV_BY_ACTOR_DIRECTOR AS
(
SELECT TRIM(MC.PID) ACTOR_PID, TRIM(MD.PID) DIRECTOR_PID, COUNT(DISTINCT TRIM(MD.MID)) AS NUM_OF_MOV
FROM M_Cast MC, M_Director MD
WHERE TRIM(MC.MID)= TRIM(MD.MID)
GROUP BY ACTOR_PID,
DIRECTOR_PID
),
NUM_OF_MOVIES_BY_YC AS
(
SELECT NM.ACTOR_PID, NM.DIRECTOR_PID, NM.NUM_OF_MOV NUM_OF_MOV_BY_YC
FROM NUM_OF_MOV_BY_ACTOR_DIRECTOR NM, YASH_CHOPRAS_PID YCP
WHERE NM.DIRECTOR_PID = YCP.PID
),
MAX_MOV_BY_OTHER_DIRECTORS AS
(
SELECT ACTOR_PID, MAX(NUM_OF_MOV) MAX_NUM_OF_MOV
FROM NUM_OF_MOV_BY_ACTOR_DIRECTOR NM, YASH_CHOPRAS_PID YCP
WHERE NM.DIRECTOR_PID <> YCP.PID 
GROUP BY ACTOR_PID
),
ACTORS_MOV_COMPARISION AS
(
SELECT NMY.ACTOR_PID,
CASE WHEN NMY.NUM_OF_MOV_BY_YC > IFNULL(NMO.MAX_NUM_OF_MOV,0) THEN 'Y' ELSE 'N' END MORE_MOV_BY_YC
FROM NUM_OF_MOVIES_BY_YC NMY
LEFT OUTER JOIN MAX_MOV_BY_OTHER_DIRECTORS NMO
ON
NMY.ACTOR_PID = NMO.ACTOR_PID 
)
SELECT DISTINCT TRIM(P.Name) ACTOR_NAME
FROM Person P
WHERE TRIM(P.PID) IN 
(
SELECT DISTINCT ACTOR_PID
FROM ACTORS_MOV_COMPARISION
WHERE MORE_MOV_BY_YC = 'Y'
)

''', con)
print(d7.head())

           ACTOR_NAME
0      Waheeda Rehman
1      Achala Sachdev
2         Yash Chopra
3          Vinod Negi
4  Chandni Jas Keerat


#### The Shahrukh number of an actor is the length of the shortest path between the actor and Shahrukh Khan in the "co-acting" graph. That is, Shahrukh Khan has Shahrukh number 0; all actors who acted in the same film as Shahrukh have Shahrukh number 1; all actors who acted in the same film as some actor with Shahrukh number 1 have Shahrukh number 2, etc. Return all actors whose Shahrukh number is 2.

In [2]:
con = sqlite3.connect('Db-IMDB.db')
d7 = pd.read_sql_query('''
WITH 
SRK_0 AS
(
    SELECT TRIM(P.PID) PID
    FROM Person P
    WHERE Trim(P.Name) like '%Shahrukh%'
),
SRK_1_MOV AS
(
    SELECT DISTINCT TRIM(mc.MID) MID, S0.PID
    FROM M_Cast mc, SRK_0 S0
    WHERE TRIM(mc.PID) = S0.PID
),
SRK_1_ACTR AS
(
    SELECT DISTINCT TRIM(mc.PID) PID
    FROM M_Cast mc, SRK_1_MOV S1M
    WHERE TRIM(mc.MID) = S1M.MID AND TRIM(mc.PID) <> S1M.PID
),
SRK_2_MOV AS
(
    SELECT DISTINCT TRIM(mc.MID) MID, S1A.PID
    FROM M_Cast mc,SRK_1_ACTR S1A
    WHERE TRIM(mc.PID) = S1A.PID
)
SELECT DISTINCT TRIM(mc.PID) PID, TRIM(P.Name) ACTOR_NAME
FROM Person P, M_Cast mc, SRK_2_MOV S2M
WHERE TRIM(mc.PID) = TRIM(P.PID) AND TRIM(mc.MID) = S2M.MID AND TRIM(mc.PID) <> S2M.PID;
''', con)
print(d7.head())

         PID             ACTOR_NAME
0  nm2951768           Freida Pinto
1  nm6467532  Caroline Christl Long
2  nm6071249          Rajeev Pahuja
3  nm3491108      Michelle Santiago
4  nm7509518         Jandre le Roux
